In [1]:
from src.modules.vin_dataclass import Vin
import src.modules.data_extractor as de
from bs4 import BeautifulSoup as BS
from pathlib import Path
from serde.json import to_json
import polars as pl

from dataclasses import dataclass
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.action_chains import ActionChains
import time

import numpy as np
import re

import requests as rq

In [23]:
def initialise_chrome(lien):
    ## LANCE CHROME ##
    service = Service(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    driver.get(lien)

    ## FERME LES COOKIES ##
    cookie_accept_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Accepter et continuer')]")
    cookie_accept_button.click()

    ## FERME POPUP ## pop-up "Inscription" apparait de manière aléatoire
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 500);")
    while True:
        try:
            time.sleep(10)
            driver.find_element(By.CLASS_NAME, 'close_modal_vinatis').click()
        except :
            continue  
        break 
    
    return driver

In [24]:
def nb_pages(driver):
    page_source = driver.page_source
    soupe = BS(page_source)
    result_nb_pages = soupe.find_all(name="span", attrs={"class": "margin-right-10"})
    return int(re.findall(r'\d+', result_nb_pages[0].text)[0])

In [25]:
def change_page(driver):
    time.sleep(2)
    bouton = driver.find_element(By.CLASS_NAME, 'icon-chevron-right')
    time.sleep(2)
    driver.execute_script("arguments[0].scrollIntoView();", bouton)
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, -500);")
    time.sleep(2)
    bouton.click()
    time.sleep(2)

In [26]:
def scraping_produits_page(driver ,vins : list()):
    page_source = driver.page_source
    soupe = BS(page_source)
        
    for (item_1, item_2, item_3, item_4, item_5) in zip(
        de.extract_result(soupe)["result_title"], 
        de.extract_result(soupe)["result_price"], 
        de.extract_result(soupe)["result_avis"],
        de.extract_result(soupe)["result_type"],
        de.extract_result(soupe)["result_title_notxt"]
    ):
        vins.append(de.extract_vin(item_1, item_2, item_3, item_4, item_5))
    
    return vins

In [21]:
def scrapeur(lien : str): 
    driver = initialise_chrome(lien)
    NOMBRE_PAGE = nb_pages(driver)
    
    i = 1
    vins = list()
    while i < 5:
        vins = scraping_produits_page(driver, vins)
        change_page(driver)
        i = i + 1
        
    driver.quit()
    
    return vins


In [27]:
vins = scrapeur("https://www.vinatis.com/achat-vin-rouge")

# RECUP ELEMENT PAGE

In [31]:
def infos(text: str):
    match = re.findall(r'[^/]+', text)
    return match

In [32]:
def url_vol(soupe : BS, vol:list()):
        ## Récupération type d'alcool ##
        text_carac = soupe.find_all(name="span", 
                attrs={"class": "ss-titre color-gray-darker taille-xs line-height-15-xs no-padding-horizontal"}
                )[0].text
        #type.append(infos(text_carac)[0])
        
        ## Récupération degré alcool ##
        if infos(text_carac)[-1].endswith("vol"):
            vol.append(infos(text_carac)[-1])
        else:
            vol.append(infos(text_carac)[-2])

        return vol
            

In [33]:
def url_adjectifs(soupe : BS, adjectif: list()):
    adjectif.append(soupe.find_all(name="div", attrs={"class": "margin-top-5 hide-xs"})[0].text)
    return adjectif

In [34]:
def url_cepages(soupe: BS, cepage : list()):
    text_cepage = soupe.find_all(name="div", 
        attrs={"class": "table-cell-css vertical-align-top padding-vertical-5 taille-xs color-gray-darker text-bold"}
        )[1].text
    cepage.append(text_cepage)
    
    return cepage

In [35]:
def extract_infos_url(vins: Vin):
    adjectif = list()
    type = list()
    vol = list()
    cepage = list()

    for i in range(len(vins)):
        requete = rq.get(url=vins[i].lien)
        soupe = BS(requete.text)
        vol = url_vol(soupe, vol)
        adjectif = url_adjectifs(soupe, adjectif)
        cepage = url_cepages(soupe, cepage)
        
    return {
        "adjectif" : adjectif,
        "type" : type,
        "vol" : vol,
        "cepage" : cepage
        }

In [36]:
def actualisation_vin(vins : Vin):
    extraction = extract_infos_url(vins)
    for i in range(len(vins)):
        vins[i].vol = extraction["vol"][i]
        vins[i].adjective = extraction["adjectif"][i]
        vins[i].cepage = extraction["cepage"][i]
    return vins

In [37]:
actualisation_vin(vins)

[Vin(name='PHI-LING 2021 - DOMAINE HORGELUS', capacity=0.75, year=2021, price=39.5, promo='-15%', prix_promo=None, note=17.0, nb_avis=1, type='VIN ROUGE', lien='https://www.vinatis.com/63736-phi-ling-2021-domaine-horgelus', vol=' 13,5 % vol', adjective=' Gourmand À partager Étonnant', cepage=' 60% Cabernet-Sauvignon, 40% Tannat'),
 Vin(name='RIOJA CRIANZA 2019 - BODEGAS BURGO VIEJO', capacity=0.75, year=2019, price=66.9, promo='-30%', prix_promo=None, note=15.8, nb_avis=82, type='VIN ROUGE', lien='https://www.vinatis.com/50860-rioja-crianza-2019-bodegas-burgo-viejo', vol=' 13,5 % vol ', adjective=" Elégance Complexité aromatique 24 mois d'élevage", cepage=' 100% Tempranillo'),
 Vin(name='VILLA DES ANGES RESERVE 2021 - JEFF CARREL', capacity=0.75, year=2021, price=66.6, promo='-30%', prix_promo=None, note=16.6, nb_avis=159, type='VIN ROUGE', lien='https://www.vinatis.com/57419-villa-des-anges-reserve-2021-jeff-carrel', vol=' 14 % vol', adjective=' BOISE EQUILIBRE VANILLE', cepage=' 60% 

In [38]:
vins

[Vin(name='PHI-LING 2021 - DOMAINE HORGELUS', capacity=0.75, year=2021, price=39.5, promo='-15%', prix_promo=None, note=17.0, nb_avis=1, type='VIN ROUGE', lien='https://www.vinatis.com/63736-phi-ling-2021-domaine-horgelus', vol=' 13,5 % vol', adjective=' Gourmand À partager Étonnant', cepage=' 60% Cabernet-Sauvignon, 40% Tannat'),
 Vin(name='RIOJA CRIANZA 2019 - BODEGAS BURGO VIEJO', capacity=0.75, year=2019, price=66.9, promo='-30%', prix_promo=None, note=15.8, nb_avis=82, type='VIN ROUGE', lien='https://www.vinatis.com/50860-rioja-crianza-2019-bodegas-burgo-viejo', vol=' 13,5 % vol ', adjective=" Elégance Complexité aromatique 24 mois d'élevage", cepage=' 100% Tempranillo'),
 Vin(name='VILLA DES ANGES RESERVE 2021 - JEFF CARREL', capacity=0.75, year=2021, price=66.6, promo='-30%', prix_promo=None, note=16.6, nb_avis=159, type='VIN ROUGE', lien='https://www.vinatis.com/57419-villa-des-anges-reserve-2021-jeff-carrel', vol=' 14 % vol', adjective=' BOISE EQUILIBRE VANILLE', cepage=' 60% 

# JSON

In [24]:
vins_json = to_json(vins)

In [25]:
file_path = "data/vins.json"

with open(file_path, "w",  encoding='utf-8') as json_file:
    json_file.write(vins_json)

In [26]:
pl.read_json(file_path)

name,capacity,year,price,promo,prix_promo,note,nb_avis,type,lien,vol,adjectifs
str,f64,i64,f64,str,f64,f64,i64,str,str,str,str
"""GIGONDAS - LE …",0.75,2019,216.7,"""-20%""",null,16.8,51,"""VIN ROUGE""","""https://www.vi…",""" 15 % vol""",""" Structure Sou…"
"""PETIT PONT ROU…",0.75,2022,34.99,"""-23%""",null,16.1,13,"""VIN ROUGE""","""https://www.vi…",""" 12,5 % vol""",""" FRUITE JUTEUX…"
"""LIRAC 2019 - X…",0.75,2019,310.4,"""-20%""",null,16.2,154,"""VIN ROUGE""","""https://www.vi…",""" 15 % vol""",""" Gourmand Frui…"
"""VILLA DES ANGE…",0.75,2021,66.6,"""-30%""",null,16.6,148,"""VIN ROUGE""","""https://www.vi…",""" 14 % vol""",""" BOISE EQUILIB…"
"""LES DARONS 202…",0.75,2022,126.8,"""-25%""",null,16.8,15,"""VIN ROUGE""","""https://www.vi…",""" 14 % vol""",""" Gourmand Meil…"
"""LE TRIPORTEUR …",0.75,2020,34.95,"""-23%""",null,15.6,97,"""VIN ROUGE""","""https://www.vi…",""" 14,5 % vol""",""" Gourmand Orig…"
"""LUPO MERAVIGLI…",0.75,2022,38.9,"""-25%""",null,16.9,20,"""VIN ROUGE""","""https://www.vi…",""" 14,5 % vol""",""" Fruité Intens…"
"""COTES DU RHONE…",0.75,2019,36.3,"""-21%""",null,15.7,145,"""VIN ROUGE""","""https://www.vi…",""" 14 % vol""",""" Fruité Velour…"
"""COTES DU RHONE…",0.75,2020,69.9,"""-16%""",null,16.9,9,"""VIN ROUGE""","""https://www.vi…",""" 15 % vol""",""" Incontournabl…"


In [1]:
import src.modules.selenium_extractor as se

In [8]:
se.build_wine("https://www.vinatis.com/rhum-ron-rum")

c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\Wine-Scraping\src\modules\selenium_extractor.py:210: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 210 of the file c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\Wine-Scraping\src\modules\selenium_extractor.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soupe = BS(requete.text)


[Vin(name='DON PAPA BAROKO - EN ETUI', capacity=0.7, year=None, price=39.9, promo=None, prix_promo=None, note=16.7, nb_avis=10, type='M', lien='https://www.vinatis.com/38929-don-papa-baroko-en-etui', vol=' 40 % vol', adjective=' Onctueux Vanillé Miel', cepage=' Nez capiteux de vanille, de miel et de délicieuses notes de fruits confits.'),
 Vin(name='RHUM ZACAPA XO - EN COFFRET FLORAL 2 VERRES', capacity=0.7, year=None, price=129.9, promo=None, prix_promo=None, note=19.3, nb_avis=11, type='M', lien='https://www.vinatis.com/48146-rhum-zacapa-xo-en-coffret-floral-2-verres', vol=' 40 % vol', adjective=' Complexe Savoureux Douceur', cepage=" Chaud, complexe et équilibré, avec des fruits secs, des notes de cacao, de bois amer. Effluves gourmandes de chêne, de caramel brûlé, de noix torréfiées, de pâte d'amande et d'écorces d'orange."),
 Vin(name='RHUM ZACAPA EDICION NEGRA - COFFRET 2 VERRES', capacity=0.7, year=None, price=79.9, promo=None, prix_promo=None, note=19.0, nb_avis=20, type='M', l